# Geiger-Nuttallのプロット
教科書に掲載されているテーブルからGeiger-Nuttallのプロットを作成する。  
事前にテーブルをテキストファイルとして作成する必要がある。  
以下の例ではそれぞれのデータをgroup1.dat, group2.dat, group3,datとしている。  
各データファイルのフォーマットは以下のように取り決める。    

```
原子番号と放出エネルギーの-1/2乗の積  崩壊確率の(自然)対数
```

各データファイルからグラフを作成します。　　

はじめにテキストデータ(またはエクセルデータ) を`pandas`データフレームに読み込みます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df1 = pd.read_csv('group1.dat', delim_whitespace=True, names=['ZE', 'logl'])
df2 = pd.read_csv('group2.dat', delim_whitespace=True, names=['ZE', 'logl'])
df3 = pd.read_csv('group3.dat', delim_whitespace=True, names=['ZE', 'logl'])

グラフを作成します。

In [ ]:
plt.plot(df1.ZE, df1.logl, 'o', label='group1')
plt.plot(df2.ZE, df2.logl, 'o', label='group2')
plt.plot(df3.ZE, df3.logl, 'o', label='group3')
plt.legend(loc='upper right')

# y切片 (C) を求めるためにフィッティング
線形フィットなので解析的に最適値は決定できます。  
しかし、グループ２，３のデータが限られているため、傾きの値はグループ１のフィッティング結果を用いることにします。

In [ ]:
from scipy.optimize import curve_fit


def f1(x, a, c):
    return a*x+c


def f2(x, a, c):
    return a*x+c


def f3(x, a, c):
    return a*x+c

グループ１に関する直線フィット

In [ ]:
par1, _ = curve_fit(f1, df1.ZE, df1.logl)

グループ２，３に対する直線フィットでは、傾きはグループ１のフィッティング結果をもちいるようにします。


In [ ]:
a_fixed = par1[0]
par2, _ = curve_fit(lambda x, c: f2(x, a_fixed, c),
                    df2.ZE, df2.logl)  # 傾きはfit1の結果！
par2 = [par1[0], par2[0]]
par3, _ = curve_fit(lambda x, c: f3(x, a_fixed, c),
                    df3.ZE, df3.logl)  # 傾きはfit1の結果！
par3 = [par1[0], par3[0]]

# データとフィッティングの結果をグラフ化

In [ ]:
ze = np.linspace(35, 47)
logl1 = f1(ze, *par1)
logl2 = f2(ze, *par2)
logl3 = f3(ze, *par3)

plt.plot(df1.ZE, df1.logl, 'o', label='group1')  # data group1
plt.plot(df2.ZE, df2.logl, 'o', label='group2')  # data group2
plt.plot(df3.ZE, df3.logl, 'o', label='group3')  # data group3
plt.plot(ze, logl1, '-', label='fit1')  # fit group1
plt.plot(ze, logl2, '-', label='fit2')  # fit group2
plt.plot(ze, logl3, '-', label='fit3')  # fit group3
plt.legend(loc='upper right')